In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
# from tensorflow_core.python.keras.models import Sequential
from tensorflow_core.python.keras.layers import Dense, Dropout
from tensorflow_core.python.keras.callbacks import EarlyStopping
from tensorflow_core.python.keras.callbacks import ModelCheckpoint

# from tensorflow_core.python.keras.wrappers.scikit_learn import KerasClassifier
# from keras.utils import np_utils
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import LabelEncoder
# from sklearn.pipeline import Pipeline
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow_core.python.keras.wrappers.scikit_learn import KerasClassifier

from tensorflow.keras.models import *
from tensorflow.keras.utils import to_categorical 

In [2]:
print('Build model...')
print('Load training data...')
df_x_train = pd.read_csv('X_train.csv', header=0, index_col = 0)
df_y_train = pd.read_csv('y_train.csv', header=0, index_col = 0)

x_train, x_test, y_train, y_test = train_test_split(df_x_train, df_y_train, test_size=0.2,stratify = df_y_train['y'])

print(y_train)

Build model...
Load training data...
      y
id     
586   1
3508  1
2486  2
2181  2
526   1
...  ..
1909  2
929   1
3211  1
3033  1
3424  1

[3840 rows x 1 columns]


In [3]:
scaler_norm = MinMaxScaler(feature_range=(-1,1))
scaler_std = StandardScaler()

print('Splitting into training and validation dataset')
X_train = x_train.values
Y_train = y_train['y'].values
X_val = x_test.values
Y_val = y_test['y'].values

X_train = scaler_std.fit_transform(X_train)
X_val = scaler_std.fit_transform(X_val)

X_train = scaler_norm.fit_transform(X_train)
X_val = scaler_norm.fit_transform(X_val)

cls_weights = class_weight.compute_class_weight('balanced', np.unique(Y_train), 
                                                Y_train)
cls_weight_dict = {0: cls_weights[0], 1: cls_weights[1], 2: cls_weights[2]}
val_sample_weights = class_weight.compute_sample_weight(cls_weight_dict, Y_val)

Y_train = to_categorical(Y_train)
Y_val = to_categorical(Y_val)

print('Load testing data...')
df_x_test = pd.read_csv('X_test.csv', header=0, index_col = 0)
X_test = df_x_test.values

X_test = scaler_std.fit_transform(X_test)

X_test = scaler_norm.fit_transform(X_test)


print(Y_val)

Splitting into training and validation dataset
Load testing data...
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [4]:
# remove outliers
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
def outlier_detection(x_train, y_train, method='isoforest'):
    if(method == 'isoforest'):
#         rng = np.random.RandomState(42)
        clf = IsolationForest(behaviour='new', max_samples=2000,
#                           random_state=rng, 
                              contamination='auto')
        clf.fit(x_train)
        indicator = clf.predict(x_train)
        index = np.where(indicator == -1)[0]
        print("detect %d outliers in training set!" % (index.shape[0]))

        x_train_clean = np.delete(x_train, index, axis=0)
        y_train_clean = np.delete(y_train, index, axis=0)

        return x_train_clean, y_train_clean
    
    elif(method == 'lof'):
        clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)
        clf.fit(x_train)
        indicator = clf.fit_predict(x_train)
        index = np.where(indicator == -1)[0]
        print("detect %d outliers in training set!" % (index.shape[0]))

        x_train_clean = np.delete(x_train, index, axis=0)
        y_train_clean = np.delete(y_train, index, axis=0)
        return x_train_clean,y_train_clean
    

X_train,Y_train = outlier_detection(X_train,Y_train,method='isoforest')
# X_train,Y_train = outlier_detection(X_train,Y_train,method='lof')

detect 59 outliers in training set!


In [5]:
# # encode class values as integers
# encoder = LabelEncoder()
# encoder.fit(y)
# encoded_Y = encoder.transform(y)
# # convert integers to dummy variables (i.e. one hot encoded)
# dummy_y = np_utils.to_categorical(encoded_Y)

# onehotencoder = OneHotEncoder(categorical_features = [0])
# x = onehotencoder.fit_transform(x).toarray()


In [6]:
# create model
def create_model(activation1='relu',activation2='sigmoid',d=0.5,n1=1024,n2=1512,n3=1512,n4=1024,n5=768,n6=256,n7=3):
    model = Sequential()
    model.add(Dense(n1, input_dim=1000, activation='relu'))
    model.add(Dropout(d))
    model.add(Dense(n2, activation=activation1))
    model.add(Dense(n3, activation=activation1))
    model.add(Dropout(d))
    model.add(Dense(n4, activation=activation1))
    model.add(Dense(n5, activation=activation1))
    model.add(Dropout(d))
    model.add(Dense(n6, activation=activation1))
    model.add(Dense(n7, activation=activation2))

    return model

In [7]:
model = create_model(activation1='relu',activation2='softmax',
             d=0.5,n1=1024,n2=1512,n3=1512,n4=1024,n5=768,n6=256,n7=3)

# Compile model
    
model.compile(loss='categorical_crossentropy', 
              optimizer='Adam', metrics=['accuracy'])
#               weighted_metrics=['accuracy'])
earlier = EarlyStopping(monitor = 'val_accuracy',mode='max',min_delta=1e-3,patience=50,verbose=1)
checkpointer = ModelCheckpoint(filepath='best.hdf5',verbose=1, save_best_only=True, monitor='val_accuracy', mode='max')

model.fit(X_train, Y_train, epochs=200, batch_size=150
          , validation_data=(X_val, Y_val,val_sample_weights)
#           , callbacks=[earlier, checkpointer])
         )


Train on 3781 samples, validate on 960 samples
Epoch 1/200
3781/3781 [==============================] - 3s 757us/sample - loss: 0.6414 - accuracy: 0.7286 - val_loss: 0.9315 - val_accuracy: 0.7292
Epoch 2/200
3781/3781 [==============================] - 2s 455us/sample - loss: 0.5226 - accuracy: 0.7762 - val_loss: 0.8643 - val_accuracy: 0.6479
Epoch 3/200
3781/3781 [==============================] - 2s 429us/sample - loss: 0.5147 - accuracy: 0.7797 - val_loss: 0.8443 - val_accuracy: 0.6687
Epoch 4/200
3781/3781 [==============================] - 2s 444us/sample - loss: 0.5148 - accuracy: 0.7778 - val_loss: 0.8609 - val_accuracy: 0.6396
Epoch 5/200
3781/3781 [==============================] - 2s 575us/sample - loss: 0.4883 - accuracy: 0.7903 - val_loss: 0.7927 - val_accuracy: 0.7073
Epoch 6/200
3781/3781 [==============================] - 2s 484us/sample - loss: 0.4612 - accuracy: 0.7985 - val_loss: 0.9312 - val_accuracy: 0.6490
Epoch 7/200
3781/3781 [==============================] - 2s

3781/3781 [==============================] - 2s 494us/sample - loss: 0.2373 - accuracy: 0.8963 - val_loss: 1.3403 - val_accuracy: 0.6448
Epoch 110/200
3781/3781 [==============================] - 2s 473us/sample - loss: 0.2151 - accuracy: 0.9111 - val_loss: 2.2891 - val_accuracy: 0.6125
Epoch 111/200
3781/3781 [==============================] - 2s 456us/sample - loss: 0.2416 - accuracy: 0.9061 - val_loss: 1.9382 - val_accuracy: 0.6177
Epoch 112/200
3781/3781 [==============================] - 2s 483us/sample - loss: 0.2122 - accuracy: 0.9148 - val_loss: 2.2452 - val_accuracy: 0.6167
Epoch 113/200
3781/3781 [==============================] - 2s 474us/sample - loss: 0.2229 - accuracy: 0.9093 - val_loss: 4.1282 - val_accuracy: 0.5885
Epoch 114/200
3781/3781 [==============================] - 2s 459us/sample - loss: 0.2146 - accuracy: 0.9095 - val_loss: 3.2526 - val_accuracy: 0.5906
Epoch 115/200
3781/3781 [==============================] - 2s 489us/sample - loss: 0.2093 - accuracy: 0.9125

In [8]:
model.save_weights('my_model_weights_crossentropy.h5')

In [9]:
# model.load_weights('best.hdf5')
Y_test = model.predict_classes(X_test)
print(Y_test)


f = open("best_cre_0.csv", "w")
f.write("id,y\n")
for i,x in enumerate(Y_test):
    f.write("{},{}\n".format(i,x))
f.close() 

[1 0 1 ... 1 0 0]


In [10]:
# using focal loss

from tensorflow.keras import backend as K
import tensorflow as tf


def focal_loss(classes_num, gamma=1, alpha=1, e=0.1):
    # classes_num contains sample number of each classes
    def focal_loss_fixed(target_tensor, prediction_tensor):
        '''
        prediction_tensor is the output tensor with shape [None, 100], where 100 is the number of classes
        target_tensor is the label tensor, same shape as predcition_tensor
        '''
        import tensorflow as tf
        from tensorflow.python.ops import array_ops
        from tensorflow.keras import backend as K

        #1# get focal loss with no balanced weight which presented in paper function (4)
        zeros = array_ops.zeros_like(prediction_tensor, dtype=prediction_tensor.dtype)
        one_minus_p = array_ops.where(tf.greater(target_tensor,zeros), target_tensor - prediction_tensor, zeros)
        FT = -1 * (one_minus_p ** gamma) * tf.math.log(tf.clip_by_value(prediction_tensor, 1e-8, 1.0))

        #2# get balanced weight alpha
        classes_weight = array_ops.zeros_like(prediction_tensor, dtype=prediction_tensor.dtype)

        total_num = float(sum(classes_num))
        classes_w_t1 = [ total_num / ff for ff in classes_num ]
        sum_ = sum(classes_w_t1)
        classes_w_t2 = [ ff/sum_ for ff in classes_w_t1 ]   #scale
        classes_w_tensor = tf.convert_to_tensor(classes_w_t2, dtype=prediction_tensor.dtype)
        classes_weight += classes_w_tensor

        alpha = array_ops.where(tf.greater(target_tensor, zeros), classes_weight, zeros)

        #3# get balanced focal loss
        balanced_fl = alpha * FT
        balanced_fl = tf.reduce_mean(balanced_fl)

        #4# add other op to prevent overfit
        # reference : https://spaces.ac.cn/archives/4493
        nb_classes = len(classes_num)
        fianal_loss = (1-e) * balanced_fl + e * K.categorical_crossentropy(K.ones_like(prediction_tensor)/nb_classes, prediction_tensor)

        return fianal_loss
    return focal_loss_fixed

    
# Recompile the model
# model.load_weights('my_model_weights.h5')
model.compile(
#     loss='categorical_crossentropy', 
    loss= focal_loss([600,3600,600],gamma=2, alpha=.25),
              optimizer='Adam', metrics=['accuracy'],
              weighted_metrics=['accuracy'])
earlier = EarlyStopping(monitor = 'val_accuracy',mode='max',min_delta=1e-3,patience=50,verbose=1)
checkpointer = ModelCheckpoint(filepath='best.hdf5',verbose=1, save_best_only=True, monitor='val_accuracy', mode='max')

# class_weight = {0 : 2., 1: 5.,2:1.}
model.fit(X_train, Y_train, epochs=200, batch_size=90,
#           class_weight=class_weight)
          validation_data=(X_val, Y_val,val_sample_weights),
#           callbacks=[earlier, checkpointer])
         )

Train on 3781 samples, validate on 960 samples
Epoch 1/200
3781/3781 [==============================] - 3s 702us/sample - loss: 0.2471 - accuracy: 0.6652 - accuracy_1: 0.6652 - val_loss: 0.1344 - val_accuracy: 0.5885 - val_accuracy_1: 0.5208
Epoch 2/200
3781/3781 [==============================] - 2s 499us/sample - loss: 0.1324 - accuracy: 0.7487 - accuracy_1: 0.7487 - val_loss: 0.1345 - val_accuracy: 0.6031 - val_accuracy_1: 0.5644
Epoch 3/200
3781/3781 [==============================] - 2s 550us/sample - loss: 0.1309 - accuracy: 0.8072 - accuracy_1: 0.8072 - val_loss: 0.1336 - val_accuracy: 0.5979 - val_accuracy_1: 0.6222
Epoch 4/200
3781/3781 [==============================] - 2s 509us/sample - loss: 0.1297 - accuracy: 0.8183 - accuracy_1: 0.8183 - val_loss: 0.1340 - val_accuracy: 0.5885 - val_accuracy_1: 0.6296
Epoch 5/200
3781/3781 [==============================] - 2s 511us/sample - loss: 0.1291 - accuracy: 0.8270 - accuracy_1: 0.8270 - val_loss: 0.1353 - val_accuracy: 0.5677 - v

Epoch 84/200
3781/3781 [==============================] - 2s 562us/sample - loss: 0.1262 - accuracy: 0.8847 - accuracy_1: 0.8847 - val_loss: 0.1358 - val_accuracy: 0.6000 - val_accuracy_1: 0.6625
Epoch 85/200
3781/3781 [==============================] - 2s 503us/sample - loss: 0.1262 - accuracy: 0.8902 - accuracy_1: 0.8902 - val_loss: 0.1406 - val_accuracy: 0.6000 - val_accuracy_1: 0.6162
Epoch 86/200
3781/3781 [==============================] - 2s 549us/sample - loss: 0.1268 - accuracy: 0.8725 - accuracy_1: 0.8725 - val_loss: 0.1385 - val_accuracy: 0.5896 - val_accuracy_1: 0.6301
Epoch 87/200
3781/3781 [==============================] - 2s 507us/sample - loss: 0.1264 - accuracy: 0.8715 - accuracy_1: 0.8715 - val_loss: 0.1411 - val_accuracy: 0.5781 - val_accuracy_1: 0.6157
Epoch 88/200
3781/3781 [==============================] - 2s 483us/sample - loss: 0.1264 - accuracy: 0.8799 - accuracy_1: 0.8799 - val_loss: 0.1400 - val_accuracy: 0.5615 - val_accuracy_1: 0.5736
Epoch 89/200
3781/37

Epoch 126/200
3781/3781 [==============================] - 2s 583us/sample - loss: 0.1255 - accuracy: 0.9069 - accuracy_1: 0.9069 - val_loss: 0.1387 - val_accuracy: 0.5833 - val_accuracy_1: 0.6319
Epoch 127/200
3781/3781 [==============================] - 2s 462us/sample - loss: 0.1264 - accuracy: 0.8953 - accuracy_1: 0.8953 - val_loss: 0.1341 - val_accuracy: 0.6219 - val_accuracy_1: 0.6792
Epoch 128/200
3781/3781 [==============================] - 2s 493us/sample - loss: 0.1264 - accuracy: 0.8900 - accuracy_1: 0.8900 - val_loss: 0.1433 - val_accuracy: 0.5469 - val_accuracy_1: 0.5787
Epoch 129/200
3781/3781 [==============================] - 2s 535us/sample - loss: 0.1260 - accuracy: 0.8945 - accuracy_1: 0.8945 - val_loss: 0.1380 - val_accuracy: 0.5865 - val_accuracy_1: 0.5986
Epoch 130/200
3781/3781 [==============================] - 2s 462us/sample - loss: 0.1258 - accuracy: 0.8998 - accuracy_1: 0.8998 - val_loss: 0.1373 - val_accuracy: 0.6292 - val_accuracy_1: 0.6593
Epoch 131/200
3

In [11]:
model.save_weights('my_model_weights_focal.h5')
Y_test = model.predict_classes(X_test)
print(Y_test)


f = open("best_focal_1.csv", "w")
f.write("id,y\n")
for i,x in enumerate(Y_test):
    f.write("{},{}\n".format(i,x))
f.close() 

[1 0 1 ... 1 0 0]


In [12]:
model.compile(loss='categorical_crossentropy', 
              optimizer='Adam', metrics=['accuracy'])
earlier = EarlyStopping(monitor = 'val_accuracy',mode='max',min_delta=1e-3,patience=50,verbose=1)
checkpointer = ModelCheckpoint(filepath='best.hdf5',verbose=1, save_best_only=True, monitor='val_accuracy', mode='max')

model.fit(X_train, Y_train, epochs=50, batch_size=90, validation_data=(X_val, Y_val,val_sample_weights)
#           , callbacks=[earlier, checkpointer])
         )

Train on 3781 samples, validate on 960 samples
Epoch 1/50
3781/3781 [==============================] - 3s 841us/sample - loss: 0.3097 - accuracy: 0.9294 - val_loss: 1.3948 - val_accuracy: 0.6521
Epoch 2/50
3781/3781 [==============================] - 2s 644us/sample - loss: 0.1729 - accuracy: 0.9410 - val_loss: 1.1287 - val_accuracy: 0.6469
Epoch 3/50
3781/3781 [==============================] - 2s 615us/sample - loss: 0.1465 - accuracy: 0.9450 - val_loss: 1.3818 - val_accuracy: 0.6458
Epoch 4/50
3781/3781 [==============================] - 2s 543us/sample - loss: 0.1615 - accuracy: 0.9389 - val_loss: 1.2903 - val_accuracy: 0.6729
Epoch 5/50
3781/3781 [==============================] - 3s 681us/sample - loss: 0.1568 - accuracy: 0.9426 - val_loss: 1.4017 - val_accuracy: 0.6521
Epoch 6/50
3781/3781 [==============================] - 2s 599us/sample - loss: 0.1432 - accuracy: 0.9458 - val_loss: 1.3619 - val_accuracy: 0.6323
Epoch 7/50
3781/3781 [==============================] - 2s 623us/

In [13]:
model.save_weights('my_model_weights_mixed_1.h5')
Y_test = model.predict_classes(X_test)
print(Y_test)


f = open("best_focal_mixed_2.csv", "w")
f.write("id,y\n")
for i,x in enumerate(Y_test):
    f.write("{},{}\n".format(i,x))
f.close() 

[1 0 1 ... 1 0 1]


In [14]:
model.compile(
#     loss='categorical_crossentropy', 
    loss= focal_loss([600,3600,600],gamma=5, alpha=.25),
              optimizer='Adam', metrics=['accuracy'],
              weighted_metrics=['accuracy'])
earlier = EarlyStopping(monitor = 'val_accuracy',mode='max',min_delta=1e-3,patience=50,verbose=1)
checkpointer = ModelCheckpoint(filepath='best.hdf5',verbose=1, save_best_only=True, monitor='val_accuracy', mode='max')

# class_weight = {0 : 2., 1: 5.,2:1.}
model.fit(X_train, Y_train, epochs=100, batch_size=50,
#           class_weight=class_weight)
          validation_data=(X_val, Y_val,val_sample_weights)
#           callbacks=[earlier, checkpointer])
         )

Train on 3781 samples, validate on 960 samples
Epoch 1/100
3781/3781 [==============================] - 4s 1ms/sample - loss: 0.1792 - accuracy: 0.5993 - accuracy_1: 0.5993 - val_loss: 0.1169 - val_accuracy: 0.5542 - val_accuracy_1: 0.6190
Epoch 2/100
3781/3781 [==============================] - 3s 844us/sample - loss: 0.1162 - accuracy: 0.7054 - accuracy_1: 0.7054 - val_loss: 0.1168 - val_accuracy: 0.5927 - val_accuracy_1: 0.6454
Epoch 3/100
3781/3781 [==============================] - 3s 862us/sample - loss: 0.1157 - accuracy: 0.8141 - accuracy_1: 0.8141 - val_loss: 0.1171 - val_accuracy: 0.6042 - val_accuracy_1: 0.6505
Epoch 4/100
3781/3781 [==============================] - 3s 781us/sample - loss: 0.1153 - accuracy: 0.8649 - accuracy_1: 0.8649 - val_loss: 0.1171 - val_accuracy: 0.6031 - val_accuracy_1: 0.6222
Epoch 5/100
3781/3781 [==============================] - 3s 783us/sample - loss: 0.1151 - accuracy: 0.8945 - accuracy_1: 0.8945 - val_loss: 0.1171 - val_accuracy: 0.6417 - val

Epoch 43/100
3781/3781 [==============================] - 3s 835us/sample - loss: 0.1147 - accuracy: 0.9185 - accuracy_1: 0.9185 - val_loss: 0.1174 - val_accuracy: 0.6521 - val_accuracy_1: 0.7019
Epoch 44/100
3781/3781 [==============================] - 3s 923us/sample - loss: 0.1146 - accuracy: 0.9267 - accuracy_1: 0.9267 - val_loss: 0.1175 - val_accuracy: 0.6458 - val_accuracy_1: 0.7037
Epoch 45/100
3781/3781 [==============================] - 4s 1ms/sample - loss: 0.1145 - accuracy: 0.9257 - accuracy_1: 0.9257 - val_loss: 0.1177 - val_accuracy: 0.6708 - val_accuracy_1: 0.6662
Epoch 46/100
3781/3781 [==============================] - 3s 903us/sample - loss: 0.1145 - accuracy: 0.9296 - accuracy_1: 0.9296 - val_loss: 0.1178 - val_accuracy: 0.6208 - val_accuracy_1: 0.6579
Epoch 47/100
3781/3781 [==============================] - 3s 805us/sample - loss: 0.1145 - accuracy: 0.9334 - accuracy_1: 0.9334 - val_loss: 0.1177 - val_accuracy: 0.6229 - val_accuracy_1: 0.6727
Epoch 48/100
3781/3781

Epoch 85/100
3781/3781 [==============================] - 3s 758us/sample - loss: 0.1145 - accuracy: 0.9320 - accuracy_1: 0.9320 - val_loss: 0.1176 - val_accuracy: 0.6687 - val_accuracy_1: 0.6861
Epoch 86/100
3781/3781 [==============================] - 3s 877us/sample - loss: 0.1146 - accuracy: 0.9193 - accuracy_1: 0.9193 - val_loss: 0.1171 - val_accuracy: 0.6677 - val_accuracy_1: 0.7111
Epoch 87/100
3781/3781 [==============================] - 3s 870us/sample - loss: 0.1144 - accuracy: 0.9360 - accuracy_1: 0.9360 - val_loss: 0.1175 - val_accuracy: 0.6604 - val_accuracy_1: 0.6870
Epoch 88/100
3781/3781 [==============================] - 3s 690us/sample - loss: 0.1144 - accuracy: 0.9344 - accuracy_1: 0.9344 - val_loss: 0.1176 - val_accuracy: 0.6552 - val_accuracy_1: 0.6824
Epoch 89/100
3781/3781 [==============================] - 3s 720us/sample - loss: 0.1146 - accuracy: 0.9212 - accuracy_1: 0.9212 - val_loss: 0.1173 - val_accuracy: 0.6365 - val_accuracy_1: 0.6741
Epoch 90/100
3781/37

In [15]:
# model.load_weights('best.hdf5')
model.save_weights('my_model_weights_mixed_2.h5')
Y_test = model.predict_classes(X_test)
print(Y_test)


f = open("best_focal_mixed_3.csv", "w")
f.write("id,y\n")
for i,x in enumerate(Y_test):
    f.write("{},{}\n".format(i,x))
f.close() 

[1 0 1 ... 1 0 0]


In [ ]:
# try subsampling
from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import RandomUnderSampler

def sub_sampling(sampling_strategy_over,sampling_strategy_down,x_train,y_train):
    model_smote = SMOTE(sampling_strategy_over)
    model_RandomUnderSampler = RandomUnderSampler(sampling_strategy_down) 
    
    x_sample_train, y_sample_train = model_smote.fit_sample(x_train,y_train)
    x_sample_train, y_sample_train =model_RandomUnderSampler.fit_sample(x_sample_train,y_sample_train)

    print('After sub-sampling:\n',pd.DataFrame(y_sample_train,columns=['y']).groupby('y').size())
    return x_sample_train,y_sample_train

sampling_strategy_over = {0:1000,2:1000}
sampling_strategy_down = {1:1000}
x_sample_train,y_sample_train = sub_sampling(sampling_strategy_over,sampling_strategy_down,x_train,y_train)

model = create_model(
             activation1='relu',activation2='sigmoid',
             d=0.5,n1=1024,n2=1512,n3=1512,n4=1024,n5=768,n6=256,n7=3)

model.fit(x_sample_train,y_sample_train, epochs=200, batch_size=90, validation_data=(X_val, Y_val,val_sample_weights))

In [ ]:
# model.load_weights('best.hdf5')
Y_test = model.predict_classes(X_test)
print(Y_test)


f = open("best_subs.csv", "w")
f.write("id,y\n")
for i,x in enumerate(Y_test):
    f.write("{},{}\n".format(i,x))
f.close() 

In [ ]:
# random search
from sklearn.model_selection import RandomizedSearchCV
# Specify parameters and distributions to sample from
n1 = [512,768,1024,1512,2048]
n2 = [256,512,768,1024,1512,2048,2560]
n3 = [256,512,768,1024,1512,2048,2560]
n4 = [128,256,512,768,1024,1512]
n5 = [128,256,512,768,1024,1512]
n6 = [64,128,256,512,768]
n7 = [3]
# batch_size = [32, 64, 96, 128, 256]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam']
# learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
activation1 = ['softplus', 'relu','exponential']
activation2 = ['softmax', 'softsign', 'tanh', 'sigmoid']
dropout_rate = [0.2, 0.5, 0.8]

model = KerasClassifier(build_fn=create_model)             
#     loss=focal_loss(alpha=1),
#              optimizer='Adam',
#              activation1='relu',activation2='softmax',
#              d=0.5,n1=1024,n2=1512,n3=1512,n4=1024,n5=768,n6=256,n7=3),
#                         epochs=200, batch_size=90))
                        # , validation_data=(X_val, Y_val,val_sample_weights))
                        # ,batch_size =batch_size,learn_rate=learn_rate)
   
# Prepare the Dict for the Search
param_dist = dict(
                n1 = n1,
                n2 = n2,
                n3 = n3,
                n4 = n4,
                n5 = n5,
                n6 = n6,
                n7 = n7,
#                 # batch_size = batch_size,
                optimizer = optimizer,
#                 # learn_rate = learn_rate,
                activation1 = activation1,
                activation2 = activation2,
                d = dropout_rate,
                loss=[focal_loss([600,3600,600])])

# Search in action!
n_iter_search = 5000 # Number of parameter settings that are sampled.
random_search = RandomizedSearchCV(estimator=model, 
                                   param_distributions=param_dist,
                                   n_iter=n_iter_search,
                                   n_jobs=-1, 
								   cv=5, 
								   verbose=1)
random_search.fit(X_train, Y_train)

# Show the results
print("Best: %f using %s" % (random_search.best_score_, random_search.best_params_))
means = random_search.cv_results_['mean_test_score']
stds = random_search.cv_results_['std_test_score']
params = random_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
0.0312 - val_weighted_acc: 0.0588
from sklearn.metrics import balanced_accuracy_score
BMAC = balanced_accuracy_score(y_true, y_pred)